# x402 Splitter Facilitator Demo (TypeScript/Deno)

Demo für den **Splitter Facilitator** - ein öffentlicher x402 Facilitator ohne Whitelist, der automatisch eine feste Fee (0.01 USDC) an den Facilitator-Wallet sendet.

## 🎯 Was ist neu?

- ✅ **Keine Whitelist** - Jeder Seller kann Zahlungen empfangen
- ✅ **Fixed Fee** - 0.01 USDC (10000 units) pro Transaktion
- ✅ **Automatische Fee-Aufteilung** - Buyer zahlt Gesamtbetrag, Splitter teilt automatisch auf:
  - `sellerAmount = totalAmount - fixedFee`
  - `facilitatorAmount = fixedFee`

## 💡 Key Concepts

**EIP3009SplitterV1 Contract:**
- Deployed auf Optimism Sepolia: `0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946`
- Nutzt EIP-3009 `transferWithAuthorization` mit automatischem Split
- Nonce-Format: `keccak256(abi.encode(seller, salt))`

In [109]:
// Setup: Imports und Konfiguration
import { load } from "https://deno.land/std@0.224.0/dotenv/mod.ts";
import { privateKeyToAccount } from "npm:viem@2/accounts";
import { createPublicClient, http, formatUnits, parseUnits } from "npm:viem@2";
import { optimism, optimismSepolia } from "npm:viem@2/chains";

// Load environment variables
const env = await load({ export: true });

const PRIVATE_KEY = env.TEST_WALLET_PRIVATE_KEY;
const PAY_TO_ADDRESS = env.NFT_WALLET_PUBLIC_KEY;

// Create account from private key
const account = privateKeyToAccount(`0x${PRIVATE_KEY}`);

console.log("🚀 x402 Facilitator Demo (TypeScript/Deno)");
console.log(`   Payer Address: ${account.address}`);
console.log(`   Recipient Address: ${PAY_TO_ADDRESS}`);

🚀 x402 Facilitator Demo (TypeScript/Deno)
   Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
   Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C


## Network-Konfiguration

Wähle zwischen **Testnet** (Optimism Sepolia) und **Mainnet** (Optimism mit echtem Geld).

⚠️ **WARNUNG**: Bei `USE_MAINNET = true` wird echtes USDC verwendet!

### Fee Structure

Der Splitter Facilitator verwendet eine **feste Fee von 0.01 USDC** pro Transaktion:

1. **Buyer zahlt:** `totalAmount = sellerAmount + fixedFee`
2. **Buyer signiert:** EIP-3009 Authorization mit `to = SPLITTER_ADDRESS` (nicht seller!)
3. **Splitter teilt auf:**
   - Seller erhält: `sellerAmount = totalAmount - fixedFee`
   - Facilitator erhält: `fixedFee = 10000 (0.01 USDC)`

Der Split passiert **automatisch on-chain** im `EIP3009SplitterV1` Contract!

### Nonce-Handling

**Standard x402:** Nonce ist random (verhindert Replay-Attacks)  
**Splitter:** Nonce = `keccak256(abi.encode(seller, salt))` 

Der Seller wird **nicht** in die EIP-3009 Authorization kodiert, sondern:
- Seller geht ins `extra.seller` Feld von `paymentRequirements`
- Facilitator berechnet beim Settlement: `nonce = keccak256(seller, salt)`
- Dies stellt sicher, dass jede seller+salt Kombination unique ist

In [110]:
// ⚠️ NETWORK SELECTION - Change this to switch between testnet and mainnet
const USE_MAINNET = false;  // Set to true for Optimism Mainnet with REAL MONEY

// Network configuration
const config = USE_MAINNET ? {
    chain: optimism,
    chainId: 10,
    caip2Network: "eip155:10" as const,
    networkName: "Optimism Mainnet",
    usdcAddress: "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85" as `0x${string}`,
    usdcName: "USD Coin",
    rpcUrl: "https://mainnet.optimism.io",
    splitterAddress: undefined  // TBD - not yet deployed on mainnet
} : {
    chain: optimismSepolia,
    chainId: 11155420,
    caip2Network: "eip155:11155420" as const,
    networkName: "Optimism Sepolia (Testnet)",
    usdcAddress: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7" as `0x${string}`,
    usdcName: "USDC",
    rpcUrl: "https://sepolia.optimism.io",
    splitterAddress: "0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946" as `0x${string}`
};

// Payment configuration
const SELLER_AMOUNT = "20000";  // $0.02 USDC (6 decimals) - what seller receives
const FIXED_FEE = "10000";      // $0.01 USDC (6 decimals) - facilitator fee
const TOTAL_AMOUNT = String(BigInt(SELLER_AMOUNT) + BigInt(FIXED_FEE));  // Total buyer pays

if (USE_MAINNET) {
    console.log(`\n🚨 WARNING: Using REAL MONEY on ${config.networkName}!`);
} else {
    console.log(`\n🧪 Using testnet: ${config.networkName}`);
}

console.log(`   Chain ID: ${config.chainId}`);
console.log(`   CAIP-2 Network: ${config.caip2Network}`);
console.log(`   USDC Address: ${config.usdcAddress}`);
console.log(`   USDC Name: ${config.usdcName}`);
console.log(`   Splitter Address: ${config.splitterAddress || 'TBD'}`);
console.log(`\n💰 Payment Breakdown:`);
console.log(`   Seller receives: ${SELLER_AMOUNT} (${Number(SELLER_AMOUNT) / 1e6} USDC)`);
console.log(`   Facilitator fee: ${FIXED_FEE} (${Number(FIXED_FEE) / 1e6} USDC)`);
console.log(`   Buyer pays total: ${TOTAL_AMOUNT} (${Number(TOTAL_AMOUNT) / 1e6} USDC)`);


🧪 Using testnet: Optimism Sepolia (Testnet)
   Chain ID: 11155420
   CAIP-2 Network: eip155:11155420
   USDC Address: 0x5fd84259d66Cd46123540766Be93DFE6D43130D7
   USDC Name: USDC
   Splitter Address: 0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946

💰 Payment Breakdown:
   Seller receives: 20000 (0.02 USDC)
   Facilitator fee: 10000 (0.01 USDC)
   Buyer pays total: 30000 (0.03 USDC)


## Facilitator Endpoints

Konfiguriere die Facilitator-URL (lokal oder deployed):

In [111]:
// Facilitator endpoint configuration
const FACILITATOR_URL = "https://x402facilitatorjccmtmdr-feefacilitator.functions.fnc.fr-par.scw.cloud";  // Deployed PoC
// const FACILITATOR_URL = "http://localhost:8081";  // Local splitter facilitator

const VERIFY_URL = `${FACILITATOR_URL}/verify`;
const SETTLE_URL = `${FACILITATOR_URL}/settle`;
const SUPPORTED_URL = `${FACILITATOR_URL}/supported`;

console.log("🚀 x402 Splitter Facilitator Endpoints:");
console.log(`   Base: ${FACILITATOR_URL}`);
console.log(`   GET  ${SUPPORTED_URL} ✅ IMPLEMENTED`);
console.log(`   POST ${VERIFY_URL} ✅ IMPLEMENTED`);
console.log(`   POST ${SETTLE_URL} ✅ IMPLEMENTED`);

🚀 x402 Splitter Facilitator Endpoints:
   Base: https://x402facilitatorjccmtmdr-feefacilitator.functions.fnc.fr-par.scw.cloud
   GET  https://x402facilitatorjccmtmdr-feefacilitator.functions.fnc.fr-par.scw.cloud/supported ✅ IMPLEMENTED
   POST https://x402facilitatorjccmtmdr-feefacilitator.functions.fnc.fr-par.scw.cloud/verify ✅ IMPLEMENTED
   POST https://x402facilitatorjccmtmdr-feefacilitator.functions.fnc.fr-par.scw.cloud/settle ✅ IMPLEMENTED


## Test /supported Endpoint

Prüfe welche Networks und Schemes der Facilitator unterstützt:

In [112]:
// Test the /supported endpoint
try {
    const supportedResponse = await fetch(SUPPORTED_URL);
    const supported = await supportedResponse.json();

    console.log(`Status Code: ${supportedResponse.status}`);
    console.log(JSON.stringify(supported, null, 2));

    if (supported.error) {
        console.log(`\n⚠️ /supported endpoint error: ${supported.error}`);
    } else {
        console.log(`\n✅ Supported Networks:`);
        for (const kind of supported.kinds) {
            console.log(`   - ${kind.network} (scheme: ${kind.scheme})`);
            console.log(`     Fee: ${kind.extra.feeDescription}`);
        }
        
        if (supported.signers && supported.signers["eip155:*"]) {
            console.log(`\n📝 Facilitator Signers:`);
            for (const signer of supported.signers["eip155:*"]) {
                console.log(`   - ${signer}`);
            }
        }
    }
} catch (error) {
    console.log(`\n⚠️ /supported endpoint error: ${error.message}`);
    console.log(`   Make sure the facilitator is running.`);
}

Status Code: 200
{
  "kinds": [
    {
      "x402Version": 2,
      "scheme": "exact-split",
      "network": "eip155:10",
      "extra": {
        "facilitatorType": "splitter",
        "splitterAddress": "0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946",
        "fixedFee": "10000",
        "feeCurrency": "USDC",
        "feeDescription": "0.01 USDC fixed fee per transaction",
        "asset": "eip155:10/erc20:0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85"
      }
    },
    {
      "x402Version": 2,
      "scheme": "exact-split",
      "network": "eip155:11155420",
      "extra": {
        "facilitatorType": "splitter",
        "splitterAddress": "0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946",
        "fixedFee": "10000",
        "feeCurrency": "USDC",
        "feeDescription": "0.01 USDC fixed fee per transaction (testnet)",
        "asset": "eip155:11155420/erc20:0x5fd84259d66Cd46123540766Be93DFE6D43130D7"
      }
    }
  ],
  "extensions": [],
  "signers": {}
}

✅ Supported Networks:
   - 

## ExactSplitEvmScheme - Custom x402 Scheme

Statt `registerExactEvmScheme` zu nutzen, implementieren wir ein **eigenes Scheme** das:

1. **Transformiert** die PaymentRequirements intern:
   - `payTo: seller` → `to: splitter` (im EIP-712)
   - `amount: sellerAmount` → `value: sellerAmount + fee`
   - Fügt `seller` und `salt` zu `extra` hinzu

2. **Signiert** die EIP-712 Authorization zum Splitter Contract

3. **Ermöglicht** Sellern, **standard x402 Code** zu verwenden!

### SchemeNetworkClient Interface

x402 v2 definiert dieses Interface für custom Schemes:

```typescript
interface SchemeNetworkClient {
  readonly scheme: string;
  createPaymentPayload(
    x402Version: number,
    paymentRequirements: PaymentRequirements,
  ): Promise<Pick<PaymentPayload, "x402Version" | "payload">>;
}
```

### Vorteile

| Aspekt | Standard "exact" | "exact-split" |
|--------|-----------------|---------------|
| Seller Code | Muss splitter-aware sein | **Standard x402!** |
| Buyer signiert zu | Seller direkt | Splitter Contract |
| Fee-Handling | Seller muss rechnen | Automatisch |

In [113]:
// Import types from x402 packages
import { x402Client } from "npm:@x402/fetch@^2.0.0";
import type { PaymentRequirements, PaymentPayload, Network } from "npm:@x402/core@^2.0.0";
import { getAddress, encodeAbiParameters, keccak256 } from "npm:viem@2";

// Splitter configuration per network
interface SplitterConfig {
    splitterAddress: `0x${string}`;
    fixedFee: bigint;
    usdcAddress: `0x${string}`;
    usdcName: string;
}

const SPLITTER_CONFIGS: Record<string, SplitterConfig> = {
    "eip155:11155420": {
        splitterAddress: "0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946",
        fixedFee: BigInt(10000),  // 0.01 USDC
        usdcAddress: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
        usdcName: "USDC"
    },
    "eip155:10": {
        splitterAddress: "0x0000000000000000000000000000000000000000",  // TBD
        fixedFee: BigInt(10000),  // 0.01 USDC
        usdcAddress: "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
        usdcName: "USD Coin"
    }
};

// EIP-712 Domain for USDC TransferWithAuthorization
function getEIP712Domain(chainId: number, usdcAddress: string, usdcName: string) {
    return {
        name: usdcName,
        version: "2",
        chainId: chainId,
        verifyingContract: usdcAddress as `0x${string}`
    };
}

// EIP-3009 TransferWithAuthorization types
const EIP3009_TYPES = {
    TransferWithAuthorization: [
        { name: "from", type: "address" },
        { name: "to", type: "address" },
        { name: "value", type: "uint256" },
        { name: "validAfter", type: "uint256" },
        { name: "validBefore", type: "uint256" },
        { name: "nonce", type: "bytes32" }
    ]
} as const;

/**
 * ExactSplitEvmScheme - Custom x402 Scheme für Fee-basierte Facilitators
 * 
 * Diese Klasse implementiert das SchemeNetworkClient Interface und ermöglicht:
 * - Seller nutzen STANDARD x402 (payTo = ihre Adresse)
 * - Scheme transformiert intern: payTo → splitter, amount += fee
 * - Buyer signiert zum Splitter Contract
 */
class ExactSplitEvmScheme {
    readonly scheme = "exact-split";
    
    constructor(
        private signer: typeof account,
        private splitterConfigs: Record<string, SplitterConfig>
    ) {}
    
    /**
     * Generiert ein random Salt für die Nonce-Berechnung
     */
    private generateSalt(): `0x${string}` {
        return `0x${Array.from({ length: 64 }, () => 
            Math.floor(Math.random() * 16).toString(16)
        ).join('')}` as `0x${string}`;
    }
    
    /**
     * Berechnet die Nonce wie im Splitter Contract:
     * nonce = keccak256(abi.encode(seller, salt))
     */
    private computeNonce(seller: string, salt: `0x${string}`): `0x${string}` {
        return keccak256(
            encodeAbiParameters(
                [{ type: "address" }, { type: "bytes32" }],
                [seller as `0x${string}`, salt]
            )
        );
    }
    
    /**
     * Extrahiert chainId aus CAIP-2 network string
     * z.B. "eip155:11155420" → 11155420
     */
    private getChainId(network: string): number {
        const parts = network.split(":");
        return parseInt(parts[1], 10);
    }
    
    /**
     * Hauptmethode: Erstellt PaymentPayload mit Splitter-Transformation
     */
    async createPaymentPayload(
        x402Version: number,
        requirements: PaymentRequirements
    ): Promise<Pick<PaymentPayload, "x402Version" | "payload">> {
        
        const network = requirements.network as string;
        const config = this.splitterConfigs[network];
        
        if (!config) {
            throw new Error(`Unsupported network for exact-split: ${network}`);
        }
        
        // 1. Get original seller from payTo
        const originalSeller = getAddress(requirements.payTo);
        
        // 2. Generate salt for unique nonce
        const salt = this.generateSalt();
        
        // 3. Compute nonce = keccak256(seller, salt)
        const nonce = this.computeNonce(originalSeller, salt);
        
        // 4. Calculate transformed amount (original + fee)
        const originalAmount = BigInt(requirements.amount);
        const totalAmount = originalAmount + config.fixedFee;
        
        // 5. Prepare authorization message
        const now = Math.floor(Date.now() / 1000);
        const validAfter = BigInt(now - 60);  // Valid from 1 minute ago
        const validBefore = BigInt(now + requirements.maxTimeoutSeconds);
        
        const authorization = {
            from: this.signer.address,
            to: config.splitterAddress,  // ← TO SPLITTER, not seller!
            value: totalAmount,
            validAfter: validAfter,
            validBefore: validBefore,
            nonce: nonce
        };
        
        // 6. Sign EIP-712 message
        const chainId = this.getChainId(network);
        const domain = getEIP712Domain(chainId, config.usdcAddress, config.usdcName);
        
        const signature = await this.signer.signTypedData({
            domain,
            types: EIP3009_TYPES,
            primaryType: "TransferWithAuthorization",
            message: authorization
        });
        
        // 7. Build payload (x402 v2 format)
        return {
            x402Version,
            payload: {
                signature,
                authorization: {
                    from: authorization.from,
                    to: authorization.to,
                    value: authorization.value.toString(),
                    validAfter: authorization.validAfter.toString(),
                    validBefore: authorization.validBefore.toString(),
                    nonce: authorization.nonce
                },
                // Splitter-specific fields (for settlement)
                seller: originalSeller,
                salt: salt,
                originalAmount: requirements.amount,
                fee: config.fixedFee.toString()
            }
        };
    }
}

// Create x402 client with our custom scheme
const splitScheme = new ExactSplitEvmScheme(account, SPLITTER_CONFIGS);
const client = new x402Client();

// Register our custom scheme for all EVM networks
client.register("eip155:*" as Network, splitScheme);

console.log("✅ x402 Client mit ExactSplitEvmScheme konfiguriert");
console.log(`   Signer address: ${account.address}`);
console.log(`   Registered scheme: ${splitScheme.scheme}`);
console.log(`   Networks: eip155:* (all EVM)`);
console.log(`\n📊 Splitter Configs:`);
for (const [network, cfg] of Object.entries(SPLITTER_CONFIGS)) {
    console.log(`   ${network}:`);
    console.log(`     Splitter: ${cfg.splitterAddress}`);
    console.log(`     Fee: ${cfg.fixedFee} (${Number(cfg.fixedFee) / 1e6} USDC)`);
}

✅ x402 Client mit ExactSplitEvmScheme konfiguriert
   Signer address: 0x553179556FC2A39e535D65b921e01fA995E79101
   Registered scheme: exact-split
   Networks: eip155:* (all EVM)

📊 Splitter Configs:
   eip155:11155420:
     Splitter: 0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946
     Fee: 10000 (0.01 USDC)
   eip155:10:
     Splitter: 0x0000000000000000000000000000000000000000
     Fee: 10000 (0.01 USDC)


## Create Payment Payload (Standard Seller Format!)

**Der große Unterschied zu vorher:**

| Vorher (Seller muss anpassen) | Jetzt (Standard!) |
|------------------------------|-------------------|
| `payTo: splitterAddress` | `payTo: sellerAddress` ✅ |
| `amount: 30000` (inkl. Fee) | `amount: 20000` (was Seller will) ✅ |
| `extra: { seller, salt }` | Nicht nötig! ✅ |

Der **ExactSplitEvmScheme** macht die Transformation automatisch:
- `payTo: seller` → `to: splitter` (in der Signatur)
- `amount: 20000` → `value: 30000` (in der Signatur)
- Generiert `seller` und `salt` intern

In [114]:
// PaymentRequirements - STANDARD FORMAT! 
// Der Seller muss NICHTS ändern - er nutzt normales x402!
// Nur das scheme ist "exact-split" statt "exact"

const paymentRequirements: PaymentRequirements = {
    scheme: "exact-split",           // ← Der einzige Unterschied!
    network: config.caip2Network,
    amount: SELLER_AMOUNT,           // ← Was der Seller will (20000)
    asset: config.usdcAddress,
    payTo: PAY_TO_ADDRESS,           // ← Seller's eigene Adresse!
    maxTimeoutSeconds: 3600,
    extra: {
        name: config.usdcName,
        version: "2"
        // Kein seller/salt nötig - ExactSplitEvmScheme macht das!
    }
};

// Build a mock PaymentRequired response (normally from server's 402 response)
const paymentRequired = {
    x402Version: 2,
    accepts: [paymentRequirements],
    resource: {
        url: "https://example.com/resource",
        description: "Test payment with splitter",
        mimeType: "application/json"
    },
    extensions: {}
};

console.log("📝 Payment Requirements (STANDARD FORMAT!):");
console.log(JSON.stringify(paymentRequirements, null, 2));

console.log(`\n✅ Seller-friendly:`);
console.log(`   - scheme: "exact-split" (statt "exact")`);
console.log(`   - payTo: ${PAY_TO_ADDRESS} (Seller's eigene Adresse!)`);
console.log(`   - amount: ${SELLER_AMOUNT} (was Seller will - OHNE Fee)`);

console.log(`\n🔄 ExactSplitEvmScheme transformiert zu:`);
console.log(`   - to: ${config.splitterAddress} (Splitter Contract)`);
console.log(`   - value: ${TOTAL_AMOUNT} (${SELLER_AMOUNT} + ${FIXED_FEE} Fee)`);
console.log(`   - seller: ${PAY_TO_ADDRESS} (in payload.seller)`);
console.log(`   - salt: random (in payload.salt)`);
console.log(`   - nonce: keccak256(seller, salt)`);

📝 Payment Requirements (STANDARD FORMAT!):
{
  "scheme": "exact-split",
  "network": "eip155:11155420",
  "amount": "20000",
  "asset": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
  "payTo": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C",
  "maxTimeoutSeconds": 3600,
  "extra": {
    "name": "USDC",
    "version": "2"
  }
}

✅ Seller-friendly:
   - scheme: "exact-split" (statt "exact")
   - payTo: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C (Seller's eigene Adresse!)
   - amount: 20000 (was Seller will - OHNE Fee)

🔄 ExactSplitEvmScheme transformiert zu:
   - to: 0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946 (Splitter Contract)
   - value: 30000 (20000 + 10000 Fee)
   - seller: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C (in payload.seller)
   - salt: random (in payload.salt)
   - nonce: keccak256(seller, salt)


In [115]:
// Create payment payload using x402 client with ExactSplitEvmScheme
// The scheme automatically:
// 1. Transforms payTo → splitter address
// 2. Adds fee to amount
// 3. Generates seller/salt fields
// 4. Computes nonce = keccak256(seller, salt)
const paymentPayload: PaymentPayload = await client.createPaymentPayload(paymentRequired as any);

console.log("✅ Payment Payload created by ExactSplitEvmScheme:");
console.log(JSON.stringify(paymentPayload, null, 2));

console.log(`\n🔐 Signature Details (TRANSFORMED!):`);
console.log(`   From: ${paymentPayload.payload?.authorization?.from}`);
console.log(`   To: ${paymentPayload.payload?.authorization?.to} ← SPLITTER!`);
console.log(`   Value: ${paymentPayload.payload?.authorization?.value} ← INKL. FEE!`);
console.log(`   Nonce: ${paymentPayload.payload?.authorization?.nonce}`);

console.log(`\n📦 Splitter-specific fields:`);
console.log(`   seller: ${paymentPayload.payload?.seller}`);
console.log(`   salt: ${paymentPayload.payload?.salt?.slice(0, 20)}...`);
console.log(`   originalAmount: ${paymentPayload.payload?.originalAmount}`);
console.log(`   fee: ${paymentPayload.payload?.fee}`);

console.log(`\n💡 Verifikation:`);
const authTo = paymentPayload.payload?.authorization?.to?.toLowerCase();
const splitterAddr = config.splitterAddress?.toLowerCase();
if (authTo === splitterAddr) {
    console.log(`   ✅ Signature ist an SPLITTER (${authTo?.slice(0, 10)}...)`);
} else {
    console.log(`   ⚠️ Signature ist an: ${authTo}`);
    console.log(`      Erwartet Splitter: ${splitterAddr}`);
}

✅ Payment Payload created by ExactSplitEvmScheme:
{
  "x402Version": 2,
  "payload": {
    "signature": "0x8cd4fd65debc298691ce3c5726edccac080e78705b326c1ad9447390b09b2f746f5f759eeac169b49a7377c460c42ae96ab926c71af1837dc76fbb60ee4645131b",
    "authorization": {
      "from": "0x553179556FC2A39e535D65b921e01fA995E79101",
      "to": "0x7e67bf96ADbf4a813DD7b0A3Ca3060a937018946",
      "value": "30000",
      "validAfter": "1767769413",
      "validBefore": "1767773073",
      "nonce": "0x823a9ebe98a21a5329304bcb2bfe90ca05d3cbd9e01e0f2fe3d7f5c8c6218e44"
    },
    "seller": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C",
    "salt": "0xa0985cf4643560cd65574da829c44b72faec512ae15288b5c912880030e0720e",
    "originalAmount": "20000",
    "fee": "10000"
  },
  "extensions": {},
  "resource": {
    "url": "https://example.com/resource",
    "description": "Test payment with splitter",
    "mimeType": "application/json"
  },
  "accepted": {
    "scheme": "exact-split",
    "network": "eip155:11

## Check USDC Balances

Vor dem Settlement prüfen wir die USDC Balances:

In [116]:
// Create public client for balance checks
const publicClient = createPublicClient({
    chain: config.chain,
    transport: http(config.rpcUrl)
});

// USDC ERC-20 balanceOf ABI
const erc20Abi = [
    {
        inputs: [{ name: "account", type: "address" }],
        name: "balanceOf",
        outputs: [{ name: "", type: "uint256" }],
        stateMutability: "view",
        type: "function"
    }
] as const;

// Splitter contract ABI (minimal - just what we need)
const splitterAbi = [
    {
        inputs: [],
        name: "facilitatorWallet",
        outputs: [{ name: "", type: "address" }],
        stateMutability: "view",
        type: "function"
    }
] as const;

// Get facilitator wallet address from splitter contract
const facilitatorWallet = await publicClient.readContract({
    address: config.splitterAddress!,
    abi: splitterAbi,
    functionName: "facilitatorWallet"
}) as `0x${string}`;

console.log(`📍 Facilitator Wallet: ${facilitatorWallet}`);

// Check balances
const payerBalance = await publicClient.readContract({
    address: config.usdcAddress,
    abi: erc20Abi,
    functionName: "balanceOf",
    args: [account.address]
});

const recipientBalance = await publicClient.readContract({
    address: config.usdcAddress,
    abi: erc20Abi,
    functionName: "balanceOf",
    args: [PAY_TO_ADDRESS]
});

const facilitatorBalance = await publicClient.readContract({
    address: config.usdcAddress,
    abi: erc20Abi,
    functionName: "balanceOf",
    args: [facilitatorWallet]
});

console.log(`\n💰 USDC Balances BEFORE Settlement on ${config.networkName}:`);
console.log(`   Payer (${account.address.slice(0, 10)}...): ${formatUnits(payerBalance, 6)} USDC`);
console.log(`   Recipient (${PAY_TO_ADDRESS.slice(0, 10)}...): ${formatUnits(recipientBalance, 6)} USDC`);
console.log(`   Facilitator (${facilitatorWallet.slice(0, 10)}...): ${formatUnits(facilitatorBalance, 6)} USDC`);

// Check if payer has enough for TOTAL_AMOUNT (not just SELLER_AMOUNT)
if (payerBalance < BigInt(TOTAL_AMOUNT)) {
    console.log(`\n⚠️ WARNING: Payer has insufficient USDC balance!`);
    console.log(`   Required: ${formatUnits(BigInt(TOTAL_AMOUNT), 6)} USDC (including ${formatUnits(BigInt(FIXED_FEE), 6)} fee)`);
    console.log(`   Has: ${formatUnits(payerBalance, 6)} USDC`);
}

📍 Facilitator Wallet: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206

💰 USDC Balances BEFORE Settlement on Optimism Sepolia (Testnet):
   Payer (0x55317955...): 0.579 USDC
   Recipient (0xAAEBC144...): 1.431 USDC
   Facilitator (0x3F8d2Fb6...): 0.02 USDC


## Verify Payment (Splitter Facilitator)

Sende den Payment Payload zur Verifizierung an den Splitter Facilitator.

### Was wird geprüft?

Der Splitter Facilitator prüft **OHNE Whitelist**:

1. ✅ **EIP-3009 Signature** - Korrekte EIP-712 Signatur
2. ✅ **Amount >= fixedFee** - Mindestens 0.01 USDC (10000 units)
3. ✅ **Token = USDC** - Nur USDC auf Optimism/Sepolia
4. ✅ **Time Window** - validAfter < now < validBefore
5. ✅ **Balance Check** - Payer hat genug USDC
6. ❌ **KEINE Whitelist** - Jeder Seller wird akzeptiert!

In [117]:
// Build verify request
const verifyRequest = {
    paymentPayload: paymentPayload,
    paymentRequirements: paymentRequirements
};

console.log("🔍 Sending verification request...");

const verifyResponse = await fetch(VERIFY_URL, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(verifyRequest)
});

const verifyResult = await verifyResponse.json();

console.log(`\n📦 Verify Response (Status ${verifyResponse.status}):`);
console.log(JSON.stringify(verifyResult, null, 2));

if (verifyResult.isValid) {
    console.log(`\n✅ Payment signature is VALID!`);
    console.log(`   Payer: ${verifyResult.payer}`);
} else {
    console.log(`\n❌ Payment signature is INVALID!`);
    console.log(`   Reason: ${verifyResult.invalidReason}`);
}

🔍 Sending verification request...

📦 Verify Response (Status 200):
{
  "isValid": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

✅ Payment signature is VALID!
   Payer: 0x553179556FC2A39e535D65b921e01fA995E79101


## Settle Payment

Führe das Settlement on-chain aus:

In [118]:
// Attempt settlement
console.log(`\n💸 Attempting Settlement...`);
console.log(`   Network: ${config.networkName}`);
console.log(`   Seller receives: $${Number(SELLER_AMOUNT) / 1e6} USDC`);
console.log(`   Total payment: $${Number(TOTAL_AMOUNT) / 1e6} USDC (incl. ${Number(FIXED_FEE) / 1e6} fee)`);
console.log(`   Recipient: ${PAY_TO_ADDRESS}`);

if (USE_MAINNET) {
    console.log(`\n🚨 WARNING: This will execute a REAL transaction with REAL MONEY!`);
    // In a real notebook, you'd have user confirmation here
}

const settleResponse = await fetch(SETTLE_URL, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(verifyRequest)
});

const settleResult = await settleResponse.json();

console.log(`\n📦 Settle Response (Status ${settleResponse.status}):`);
console.log(JSON.stringify(settleResult, null, 2));

let txHash: string | null = null;

if (settleResult.success) {
    txHash = settleResult.transaction;
    console.log(`\n🎉 Settlement successful!`);
    console.log(`   Transaction: ${txHash}`);
    console.log(`   Network (Response): ${settleResult.network}`);
} else {
    console.log(`\n❌ Settlement failed`);
    console.log(`   Reason: ${settleResult.errorReason || "unknown"}`);
}


💸 Attempting Settlement...
   Network: Optimism Sepolia (Testnet)
   Seller receives: $0.02 USDC
   Total payment: $0.03 USDC (incl. 0.01 fee)
   Recipient: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

📦 Settle Response (Status 200):
{
  "success": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "0x2318cf08703fb481eddc4bfede421a6c278fb511b560358852f6a9c90ddd199d",
  "network": "eip155:11155420"
}

🎉 Settlement successful!
   Transaction: 0x2318cf08703fb481eddc4bfede421a6c278fb511b560358852f6a9c90ddd199d
   Network (Response): eip155:11155420


## Verify Balances After Settlement

Prüfe die Balances nach dem Settlement, um zu verifizieren:
- ✅ Seller erhielt: `SELLER_AMOUNT` (0.02 USDC)
- ✅ Facilitator erhielt: `FIXED_FEE` (0.01 USDC)
- ✅ Payer zahlte: `TOTAL_AMOUNT` (0.03 USDC)

In [121]:
// Check balances AFTER settlement
const payerBalanceAfter = await publicClient.readContract({
    address: config.usdcAddress,
    abi: erc20Abi,
    functionName: "balanceOf",
    args: [account.address]
});

const recipientBalanceAfter = await publicClient.readContract({
    address: config.usdcAddress,
    abi: erc20Abi,
    functionName: "balanceOf",
    args: [PAY_TO_ADDRESS]
});

const facilitatorBalanceAfter = await publicClient.readContract({
    address: config.usdcAddress,
    abi: erc20Abi,
    functionName: "balanceOf",
    args: [facilitatorWallet]
});

console.log(`\n💰 USDC Balances AFTER Settlement on ${config.networkName}:`);
console.log(`   Payer (${account.address.slice(0, 10)}...): ${formatUnits(payerBalanceAfter, 6)} USDC`);
console.log(`   Recipient (${PAY_TO_ADDRESS.slice(0, 10)}...): ${formatUnits(recipientBalanceAfter, 6)} USDC`);
console.log(`   Facilitator (${facilitatorWallet.slice(0, 10)}...): ${formatUnits(facilitatorBalanceAfter, 6)} USDC`);

// Calculate deltas
const payerDelta = payerBalance - payerBalanceAfter;
const recipientDelta = recipientBalanceAfter - recipientBalance;
const facilitatorDelta = facilitatorBalanceAfter - facilitatorBalance;

console.log(`\n📊 Balance Changes:`);
console.log(`   Payer spent: ${formatUnits(payerDelta, 6)} USDC`);
console.log(`   Recipient gained: ${formatUnits(recipientDelta, 6)} USDC`);
console.log(`   Facilitator gained: ${formatUnits(facilitatorDelta, 6)} USDC`);

// Verify correctness
console.log(`\n✅ Verification:`);
const expectedSellerAmount = BigInt(SELLER_AMOUNT);
const expectedFee = BigInt(FIXED_FEE);
const expectedTotal = BigInt(TOTAL_AMOUNT);

if (recipientDelta === expectedSellerAmount) {
    console.log(`   ✅ Seller received correct amount: ${formatUnits(expectedSellerAmount, 6)} USDC`);
} else {
    console.log(`   ❌ Seller amount mismatch! Expected: ${formatUnits(expectedSellerAmount, 6)}, Got: ${formatUnits(recipientDelta, 6)}`);
}

if (facilitatorDelta === expectedFee) {
    console.log(`   ✅ Facilitator received correct fee: ${formatUnits(expectedFee, 6)} USDC`);
} else {
    console.log(`   ❌ Fee mismatch! Expected: ${formatUnits(expectedFee, 6)}, Got: ${formatUnits(facilitatorDelta, 6)}`);
}

if (payerDelta === expectedTotal) {
    console.log(`   ✅ Payer spent correct total: ${formatUnits(expectedTotal, 6)} USDC`);
} else {
    console.log(`   ❌ Total mismatch! Expected: ${formatUnits(expectedTotal, 6)}, Got: ${formatUnits(payerDelta, 6)}`);
}

if (recipientDelta + facilitatorDelta === payerDelta) {
    console.log(`   ✅ Conservation of funds: Seller + Fee = Total paid`);
} else {
    console.log(`   ❌ Funds don't add up!`);
}


💰 USDC Balances AFTER Settlement on Optimism Sepolia (Testnet):
   Payer (0x55317955...): 0.549 USDC
   Recipient (0xAAEBC144...): 1.451 USDC
   Facilitator (0x3F8d2Fb6...): 0.03 USDC

📊 Balance Changes:
   Payer spent: 0.03 USDC
   Recipient gained: 0.02 USDC
   Facilitator gained: 0.01 USDC

✅ Verification:
   ✅ Seller received correct amount: 0.02 USDC
   ✅ Facilitator received correct fee: 0.01 USDC
   ✅ Payer spent correct total: 0.03 USDC
   ✅ Conservation of funds: Seller + Fee = Total paid


## Block Explorer & Cross-Chain Validation

Wichtig: Wir nutzen das `network` Feld aus der Server-Response, nicht lokale Variablen!

In [120]:
// Display transaction on block explorer
// ✅ WICHTIG: Wir nutzen das 'network' Feld aus der Server-Response!

if (txHash) {
    const responseNetwork = settleResult.network || "";
    
    // Cross-Chain Validation: Does response match expected network?
    if (responseNetwork && responseNetwork !== config.caip2Network) {
        console.log(`⚠️ WARNING: Network mismatch!`);
        console.log(`   Expected: ${config.caip2Network} (${config.networkName})`);
        console.log(`   Response: ${responseNetwork}`);
        console.log(`   Transaction may have been executed on a different network!`);
    }
    
    // Block Explorer URL mapping from CAIP-2 network
    const networkToExplorer: Record<string, string> = {
        "eip155:10": "https://optimistic.etherscan.io/tx/",
        "eip155:11155420": "https://sepolia-optimism.etherscan.io/tx/",
        "eip155:8453": "https://basescan.org/tx/",
        "eip155:84532": "https://sepolia.basescan.org/tx/",
    };
    
    const explorerBase = networkToExplorer[responseNetwork] || networkToExplorer[config.caip2Network];
    const explorerUrl = explorerBase ? `${explorerBase}${txHash}` : null;
    
    console.log(`\n🔍 Block Explorer:`);
    if (explorerUrl) {
        console.log(`   ${explorerUrl}`);
    }
    console.log(`\n📊 Transaction Details:`);
    console.log(`   • Hash: ${txHash}`);
    console.log(`   • Network (Response): ${responseNetwork}`);
    console.log(`   • Network (Expected): ${config.caip2Network}`);
    console.log(`   • Token: USDC at ${config.usdcAddress}`);
} else {
    console.log(`\n⚠️ No transaction hash available`);
}


🔍 Block Explorer:
   https://sepolia-optimism.etherscan.io/tx/0x2318cf08703fb481eddc4bfede421a6c278fb511b560358852f6a9c90ddd199d

📊 Transaction Details:
   • Hash: 0x2318cf08703fb481eddc4bfede421a6c278fb511b560358852f6a9c90ddd199d
   • Network (Response): eip155:11155420
   • Network (Expected): eip155:11155420
   • Token: USDC at 0x5fd84259d66Cd46123540766Be93DFE6D43130D7


## Summary

Dieses Notebook demonstriert den kompletten x402 v2 Payment Flow:

1. ✅ **Setup**: x402Client mit ExactEvmScheme registriert
2. ✅ **Payment Creation**: Automatische EIP-712 Signatur via `createPaymentPayload()`
3. ✅ **Verification**: Signatur beim Facilitator verifiziert
4. ✅ **Settlement**: On-chain Transaktion ausgeführt
5. ✅ **Cross-Chain Validation**: Network aus Response validiert

**Vorteile der TypeScript/x402 Version:**
- Typsicherheit durch TypeScript
- Offizielle x402 Packages von Coinbase
- Automatisches Payment-Handling
- Korrekte EIP-712 Domain-Konfiguration durch die Library